In [2]:
# gpu support
# may need to install nvidia drivers etc ref: https://lightgbm.readthedocs.io/en/latest/GPU-Tutorial.html
!git clone --recursive https://github.com/microsoft/LightGBM
!cd LightGBM
!mkdir build
!cd build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!cd ..


/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
fatal: destination path 'LightGBM' already exists and is not an empty directory.
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
mkdir: cannot create directory ‘build’: File exists
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition 

In [3]:
!pip install lightgbm
!pip install dask[complete]
!pip install sklearn

/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: switchml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `switchml'
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ubuntu/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
/bin/bash: module: line 1: syntax error: unexpected end of fil

In [1]:
import numpy as np
import lightgbm as lgb
import pandas as pd
import dask.dataframe as dd


In [3]:
from distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
# client.restart()
# client.close()
# client.shutdown()

/home/ubuntu/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42311 instead
  warnings.warn(


In [4]:
%%time

# dask
# https://youtu.be/lPcX2xev0ho

training_features = dd.read_csv("./originalDataset/train_data.csv", blocksize=25e6)
training_label = dd.read_csv("./originalDataset/train_labels.csv", blocksize=25e6)

training_merged = training_features.merge(
    training_label, 
    how="outer", 
    on=["customer_ID"]
)

# training_merged.compute()
# training_merged.head(n=5)





CPU times: user 361 ms, sys: 13.6 ms, total: 375 ms
Wall time: 375 ms


In [5]:
%%time
# https://towardsdatascience.com/parallelizing-feature-engineering-with-dask-3db88aec33b7
# feature engineering
categorical_feature_array=[
        "B_30", "B_38", "D_114", 
        "D_116", "D_117", "D_120",
        "D_126", "D_64",
        "D_66", "D_68"
]
# removed column D_63 because had ML algo (fit) couldn't understand the column & couldn't properly change type 


def feature_engineering(dataframe):
        # dataframe["D_63"] = dataframe["D_63"].astype("category")
        # dataframe["D_64"] = dataframe["D_64"].astype("category")

        dataframe[categorical_feature_array] = dataframe[categorical_feature_array].astype("category")

        
        dataframe["S_2"] = dd.to_datetime(dataframe["S_2"])
        dataframe["month"] = dataframe["S_2"].dt.month
        dataframe["day"] = dataframe["S_2"].dt.dayofweek
        dataframe["year"] = dataframe["S_2"].dt.year
        return dataframe.drop(columns=["S_2", "D_63"])

training_merged = feature_engineering(training_merged)

# learn dask
# https://youtu.be/lPcX2xev0ho

CPU times: user 888 ms, sys: 93 ms, total: 981 ms
Wall time: 875 ms


In [6]:
# print(training_merged[["D_63","D_64"]].dtypes)
# print(training_merged["D_64"].head(n=5))
# training_merged["D_63"].nunique().compute()
# training_merged["D_64"].nunique().compute()

In [7]:
%%time
# features = training_merged[training_merged.columns[training_merged.columns!="target"]]
# label = training_merged["target"]

# must create training set, validation set, & test set using training_merged to train, validate, & test model



training_set, validation_set, test_set = training_merged.random_split([0.8, 0.1, 0.1], random_state=123)

train_x = training_set.drop(columns=["target","customer_ID"])
train_y = training_set["target"]

val_x = validation_set.drop(columns=["target","customer_ID"])
val_y = validation_set["target"]

test_x = test_set.drop(columns=["target","customer_ID"])
test_y = test_set["target"]


CPU times: user 178 ms, sys: 10.9 ms, total: 189 ms
Wall time: 167 ms


In [8]:
%%time

# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html
# https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html#dask
# https://www.analyticsvidhya.com/blog/2021/08/complete-guide-on-how-to-use-lightgbm-in-python/
# model development



# https://lightgbm.readthedocs.io/en/latest/Parameters.html
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
# https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#for-better-accuracy



params = {
    # change accuracy, use dart bosoting type?
    "num_leaves" : 31,
    "max_bin" : 63,
    "learning_rate" : 0.1,
    "boosting_type" : "goss",

    # change overfitting
    "max_depth" : -1,
    "min_split_gain" : 0,
    "min_child_samples" : 20,
    "reg_alpha" : 0,
    "reg_lambda" : 0,
    "early_stopping" : 5,
    "feature_pre_filter" : True,

    # 20x weight on negative
    "scale_pos_weight" : 0.05,

    # gpu
    "device_type" : "gpu",
    "gpu_platform_id" : 0,
    "gpu_device_id" : 0,

    # misc
    "verbose" : -1,
    "objective" : "binary",
    "n_jobs" : -1,
    "random_state" : 42
}



model = lgb.DaskLGBMClassifier(client=client, **params)



CPU times: user 62 µs, sys: 10 µs, total: 72 µs
Wall time: 79.2 µs


In [9]:
%%time

model.fit(
    train_x,
    train_y,
    eval_set=[(val_x, val_y),(train_x, train_y)],
    eval_metric="logloss"
    # ,categorical_feature=categorical_feature_array
)

print(f"Training accuracy: {model.score(train_x, train_y)}")
print(f"Validation accuracy: {model.score(val_x, val_y)}")
print(f"Test accuracy: {model.score(test_x, test_y)}")

/home/ubuntu/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/lightgbm/dask.py:525: UserWarning: Parameter n_jobs will be ignored.
  _log_warning(f"Parameter {param_alias} will be ignored.")
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [ ]:
# tuning https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

In [ ]:
%%time
# https://www.analyticsvidhya.com/blog/2021/08/complete-guide-on-how-to-use-lightgbm-in-python/
# Additional
from sklearn.metrics import classification_report
lgb.plot_importance(model)
lgb.plot_metric(model)
print(classification_report(test_y,model.predict(test_x)))

: 

In [ ]:
%%time
# make predictions on test_data features and reference sample_submission for format
test_features_with_ID = dd.read_csv("./originalDataset/test_data.csv", blocksize=25e6)

# changes features to match what was used to train model
test_features_with_ID = feature_engineering(test_features_with_ID)

test_features = test_features_with_ID.drop(columns=["customer_ID"])
test_features.compute()
pred_y = model.predict(test_features)
final_submit_df = test_features_with_ID["customer_ID"]
final_submit_df["prediction"] = pred_y
final_submit_df.to_csv("final_submit.csv", single_file=True)

# in cli
# kaggle competitions submit -c [COMPETITION] -f [FILE] -m [MESSAGE]
!kaggle competitions submit -c amex-default-prediction -f final_submit.csv -m "first submit"

In [ ]:
# pickle model when done?